In [1]:
%pip install langchain langchain-community langchain-huggingface neo4j pandas spacy trans
!python -m spacy download en_core_web_sm

Note: you may need to restart the kernel to use updated packages.


Python was not found; run without arguments to install from the Microsoft Store, or disable this shortcut from Settings > Apps > Advanced app settings > App execution aliases.


In [2]:
from langchain_community.graphs import Neo4jGraph
from neo4j import GraphDatabase

In [3]:
config = {}

with open('config.txt', 'r') as file:
    for line in file:
        if "=" in line:
            key, value = line.split('=', 1)
            config[key.strip()] = value.strip()

uri = config.get('URI')
username = config.get('USERNAME')
password = config.get('PASSWORD')
driver = GraphDatabase.driver(uri, auth=(username, password))
print("Connected to Neo4j database")

Connected to Neo4j database


In [4]:
# Connect using the LangChain wrapper
graph = Neo4jGraph(
    url=uri,
    username=username,
    password=password,
    refresh_schema= False
)
# Ensure the connection is working by running a quick query (optional)
print(graph.query("MATCH (s:Season) RETURN s"))

C:\Users\Habiba\AppData\Local\Temp\ipykernel_12880\1507928951.py:2: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the `langchain-neo4j package and should be used instead. To use it run `pip install -U `langchain-neo4j` and import as `from `langchain_neo4j import Neo4jGraph``.
  graph = Neo4jGraph(


[{'s': {'season_name': '2021-22'}}, {'s': {'season_name': '2022-23'}}]


In [69]:
import spacy
import re

nlp = spacy.load("en_core_web_sm")
# Load the kb from the graph database (optional enhancement)
def load_fpl_kb(graph: Neo4jGraph) -> dict:
    kb = {
        "players": [],
        "teams": [],
        "positions": ["gk","gkp", "def", "mid", "fwd", "goalkeeper","goalkeepers" "defender", "midfielder", "forward","defenders","midfielders","forwards"],
        "stats": {}
    }

    # Load players
    player_results = graph.query("MATCH (p:Player) RETURN p.player_name AS name")
    kb["players"] = [record["name"] for record in player_results]

    # Load teams
    team_results = graph.query("MATCH (t:Team) RETURN t.name AS name")
    kb["teams"] = [record["name"] for record in team_results]

    # Load stats mapping
    kb["stats"] = {
        "points": "total_points",
        "goals": "goals_scored",
        "assists": "assists",
        "minutes": "minutes",
        "bonus": "bonus",
        "influence": "influence",
        "creativity": "creativity",
        "threat": "threat",
        "ict": "ict_index",
        "clean sheets": "clean_sheets",
        "form": "form"
    }

    return kb

In [73]:
FPL_KB = load_fpl_kb(graph)

In [7]:
import transformers
import torch
from transformers import pipeline
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli")

C:\Users\Habiba\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Device set to use cpu


In [8]:
INTENTS = {
    "fixture_details": [
        "fixture", "fixtures", "when do", "when does", "when is", "play next", 
        "next match", "kickoff", "schedule", "upcoming match", "future match"
    ],

    "best_players_by_metric": [
        "top", "best", "highest", "leader", "rank", "ranking", 
        "top scorer", "top assist", "highest points", "most points", "stat leaders", "top players","best forward",
        "best midfielder", "best defender", "best goalkeeper","top number","best number"
    ],
    "player_or_team_performance": [
        "how did", "performance", "stats", "statistics", "record", "scored", 
        "assists", "goals", "points", "clean sheets", "how many", 
        "results","compare", "vs", "versus", "better than", "head to head", "compare stats", "comparison","more than","compare player1 and player2"
    ]
}

In [9]:
def classify_fpl_intents(query: str):
    """
    Returns the highest confidence intent
    """
    candidate_labels = list(INTENTS.keys())
    result = classifier(query, candidate_labels, multi_label=True)

    return result["labels"][0]  # the top intent

In [75]:
ENTITY_LOOKUP = {}

def add_to_lookup(terms, category):
    for item in terms:
        # If it's a dict (like stats), the item is the key, canonical is the value
        if isinstance(terms, dict):
            value = terms[item]
            key = item
        else:
            value = item.title()
            key = item
        
        ENTITY_LOOKUP[key.lower()] = (category, value)
add_to_lookup(FPL_KB["players"], "player")
add_to_lookup(FPL_KB["teams"], "team")
add_to_lookup(FPL_KB["positions"], "position")
add_to_lookup(FPL_KB["stats"], "stat")

In [ ]:
def extract_fpl_entities(query: str) -> dict:
    """
    Extract entities from FPL query with improved accuracy and validation
    """
    doc = nlp(query)
    entities = {
        "stat_type": "total_points",  # Default fallback
        "season": "2022-23",  # Default season
        "limit": 10  # Default limit
    }
    
    query_lower = query.lower()
    
    # ============================================================================
    # STEP 1: Extract using spaCy + lookup
    # ============================================================================
    
    # Track which players/teams we've seen to handle comparisons
    seen_players = []
    seen_teams = []
    
    for token in doc:
        text = token.text.lower()
        lemma = token.lemma_.lower()
    

        match = ENTITY_LOOKUP.get(text) or ENTITY_LOOKUP.get(lemma)
        
        if match:
            category, value = match
            
            # Handle Players
            if category == "player":
                if value not in seen_players:
                    seen_players.append(value)
                    if "player1" not in entities:
                        entities["player1"] = value
                        entities["player_name"] = value
                    elif "player2" not in entities:
                        entities["player2"] = value
            
            # Handle Teams
            elif category == "team":
                if value not in seen_teams:
                    seen_teams.append(value)
                    if "team1" not in entities:
                        entities["team1"] = value
                        entities["team_name"] = value
                    elif "team2" not in entities:
                        entities["team2"] = value
            
            # Handle Positions
            elif category == "position":
                    # Normalize Aliases
                    norm = value.upper()
                    if "MID" in norm: norm = "MID"
                    elif "FWD" in norm or "FORWARD" in norm: norm = "FWD"
                    elif "DEF" in norm: norm = "DEF"
                    elif "GK" in norm or "GKP" in norm or "GOALKEEPER" in norm: norm = "GKP"
                    entities["position"] = norm
            # Handle Stats
            elif category == "stat":
                entities["stat_type"] = value
    
    # ============================================================================
    # STEP 2: Multi-word entity extraction (e.g., "Mohamed Salah")
    # ============================================================================
    
    # Extract multi-word player names
    for player in FPL_KB["players"]:
        if player.lower() in query_lower:
            if player not in seen_players:
                seen_players.append(player)
                if "player1" not in entities:
                    entities["player1"] = player
                    entities["player_name"] = player
                elif "player2" not in entities:
                    entities["player2"] = player
    
    # Extract multi-word team names (e.g., "Manchester City")
    for team in FPL_KB["teams"]:
        if team.lower() in query_lower:
            if team not in seen_teams:
                seen_teams.append(team)
                if "team1" not in entities:
                    entities["team1"] = team
                    entities["team_name"] = team
                elif "team2" not in entities:
                    entities["team2"] = team
    
    # Extract multi-word stats (e.g., "clean sheets")
    for stat_key, stat_value in FPL_KB["stats"].items():
        if stat_key in query_lower:
            entities["stat_type"] = stat_value
            break
    
    # ============================================================================
    # STEP 3: Regex extraction for structured patterns
    # ============================================================================
    
    # Extract Gameweek
    gw_match = re.search(r"(?:gw|gameweek|game week)\s*(\d+)", query_lower)
    if gw_match:
        entities["gw_number"] = int(gw_match.group(1))
    
    # Extract Season (2022-23 format)
    season_match = re.search(r"(20\d{2}-\d{2})", query_lower)
    if season_match:
        entities["season"] = season_match.group(1)
    else:
        # Try alternative format: "season 2022" or "in 2022"
        year_match = re.search(r"(?:season|year|in)\s*(20\d{2})", query_lower)
        if year_match:
            year = year_match.group(1)
            next_year = str(int(year) + 1)[-2:]
            entities["season"] = f"{year}-{next_year}"
    
    # Extract "top N" or "best N"
    limit_match = re.search(r"(?:top|best|first)\s*(\d+)", query_lower)
    if limit_match:
        entities["limit"] = int(limit_match.group(1))
    
    # Extract current gameweek for recommendations
    if any(word in query_lower for word in ["recommend", "suggest", "current", "now", "right now"]):
        # If no specific GW mentioned, assume current GW context
        if "current_gw" not in entities and "gw_number" in entities:
            entities["current_gw"] = entities["gw_number"]
        elif "current_gw" not in entities:
            entities["current_gw"] = 20  # Default mid-season
    
    # Extract minimum value filters (e.g., "more than 10 points")
    filter_match = re.search(r"(?:more than|over|at least|minimum)\s*(\d+)", query_lower)
    if filter_match:
        entities["filter_value"] = int(filter_match.group(1))
    
    return entities

In [33]:
def get_fpl_cypher_query(intent: str, entities: dict) -> str:
    """
    Generate Cypher query based on FPL intents + extracted entities
    """

    player1 = entities.get("player1")
    player2 = entities.get("player2")
    team1 = entities.get("team1")
    team2 = entities.get("team2")
    stat = entities.get("stat_type", "total_points")
    gw = entities.get("gw_number")
    limit = entities.get("limit", 10)
    season = entities.get("season", "2022-23")
    

    # ----------------------------------------------------------------------
    # 1) PERFORMANCE: Single Player
    # ----------------------------------------------------------------------
    if intent == "player_or_team_performance" and player1 and not player2:
        if gw:
            return f"""
                MATCH (:Season {{season_name:'{season}'}})-[:HAS_GW]->(g:Gameweek {{GW_number:{gw}}})-[:HAS_FIXTURE]->(f:Fixture)
                MATCH (p:Player {{player_name:'{player1}'}})-[pi:PLAYED_IN]->(f)
                RETURN p.player_name AS player, pi.{stat} AS {stat}, g.GW_number AS gameweek
        
            """
        
        else:
            return f"""
                MATCH (:Season {{season_name:'{season}'}})-[:HAS_GW]->(g)-[:HAS_FIXTURE]->(f)
                MATCH (p:Player {{player_name:'{player1}'}})-[pi:PLAYED_IN]->(f)
                RETURN p.player_name AS player, SUM(pi.{stat}) AS total_{stat}, '{season}' AS season
            """

    # ----------------------------------------------------------------------
    # 2) PERFORMANCE: Single Team Performance summary
    # ----------------------------------------------------------------------
    if intent == "player_or_team_performance" and team1 and not team2:
        if gw:
            return f"""
                MATCH (:Season {{season_name:'{season}'}})-[:HAS_GW]->(g:Gameweek {{GW_number:{gw}}})-[:HAS_FIXTURE]->(f:Fixture)
                WHERE f.home_team = '{team1}' OR f.away_team = '{team1}'
                MATCH (p:Player)-[pi:PLAYED_IN]->(f)
                RETURN '{team1}' AS team, SUM(pi.{stat}) AS total_{stat}, g.GW_number AS gameweek
            """
        else:
            return f"""
                MATCH (:Season {{season_name:'{season}'}})-[:HAS_GW]->(g)-[:HAS_FIXTURE]->(f)
                WHERE f.home_team = '{team1}' OR f.away_team = '{team1}'
                MATCH (p:Player)-[pi:PLAYED_IN]->(f)
                RETURN '{team1}' AS team, SUM(pi.{stat}) AS total_{stat}
            """

    # ----------------------------------------------------------------------
    # 3) PERFORMANCE: Compare Two Players
    # ----------------------------------------------------------------------
    if intent == "player_or_team_performance" and player1 and player2:
        return f"""
            MATCH (:Season {{season_name:'{season}'}})-[:HAS_GW]->(g)-[:HAS_FIXTURE]->(f)
            MATCH (p:Player)-[pi:PLAYED_IN]->(f)
            WHERE p.player_name IN ['{player1}', '{player2}']
            RETURN p.player_name AS player, SUM(pi.{stat}) AS total_{stat}
        """

    # ----------------------------------------------------------------------
    # 4) PERFORMANCE: Compare Two Teams
    # ----------------------------------------------------------------------
    if intent == "player_or_team_performance" and team1 and team2:
        return f"""
            MATCH (:Season {{season_name:'{season}'}})-[:HAS_GW]->(g)-[:HAS_FIXTURE]->(f)
            WHERE f.home_team IN ['{team1}', '{team2}'] OR f.away_team IN ['{team1}', '{team2}']
            MATCH (p:Player)-[pi:PLAYED_IN]->(f)
            RETURN CASE 
                WHEN f.home_team = '{team1}' OR f.away_team = '{team1}' THEN '{team1}'
                ELSE '{team2}' END AS team,
                SUM(pi.{stat}) AS total_{stat}
        """

    # ----------------------------------------------------------------------
    # 5) FIXTURE: Next Fixture by Team
    # ----------------------------------------------------------------------
    if intent == "fixture_details" and team1 and gw and not team2:
        return f"""
             MATCH (:Season {{season_name:'{season}'}})-[:HAS_GW]->(g:Gameweek {{GW_number:{gw}}})-[:HAS_FIXTURE]->(f:Fixture)-[:HAS_HOME_TEAM]->(h:Team),
            (f)-[:HAS_AWAY_TEAM]->(a:Team)
            WHERE h.name = '{team1}' OR a.name = '{team1}'
            RETURN f.fixture_number AS fixture, f.kickoff_time, h, a
            ORDER BY f.kickoff_time ASC LIMIT 1
        """
    
    

    # ----------------------------------------------------------------------
    # 6) FIXTURE: Next Fixture involving Two Teams (Head-to-head)
    # ----------------------------------------------------------------------
    if intent == "fixture_details" and team1 and team2 and gw:
        return f"""
            MATCH (:Season {{season_name:'{season}'}})-[:HAS_GW]->(g:Gameweek {{GW_number:{gw}}})-[:HAS_FIXTURE]->(f:Fixture)
            WHERE (f.home_team='{team1}' AND f.away_team='{team2}')
            OR (f.home_team='{team2}' AND f.away_team='{team1}')
            RETURN f.fixture_number AS fixture, f.kickoff_time, f.home_team, f.away_team
            ORDER BY f.kickoff_time ASC LIMIT 1
        """

    # ----------------------------------------------------------------------
    # 7) BEST PLAYERS BY METRIC: Overall
    # ----------------------------------------------------------------------
    if intent == "best_players_by_metric" and not entities.get("position"):
        return f"""
            MATCH (:Season {{season_name:'{season}'}})-[:HAS_GW]->(g)-[:HAS_FIXTURE]->(f)
            MATCH (p:Player)-[pi:PLAYED_IN]->(f)
            RETURN p.player_name AS player, SUM(pi.{stat}) AS total_{stat}
            ORDER BY total_{stat} DESC LIMIT {limit}
        """

    # ----------------------------------------------------------------------
    # 8) BEST PLAYERS BY METRIC AND POSITION
    # ----------------------------------------------------------------------
    if intent == "best_players_by_metric" and entities.get("position"):
        position = entities["position"]
        return f"""
            MATCH (p:Player)-[:PLAYS_AS]->(pos:Position {{name:'{position}'}})
            MATCH (:Season {{season_name:'{season}'}})-[:HAS_GW]->(g)-[:HAS_FIXTURE]->(f)
            MATCH (p)-[pi:PLAYED_IN]->(f)
            RETURN p.player_name AS player, SUM(pi.{stat}) AS total_{stat}, pos.name AS position
            ORDER BY total_{stat} DESC LIMIT {limit}
        """

    # ----------------------------------------------------------------------
    # 9) BEST PLAYERS — FILTER WHERE STAT ABOVE VALUE
    # ----------------------------------------------------------------------
    if intent == "best_players_by_metric" and entities.get("filter_value"):
        val = entities["filter_value"]
        return f"""
            MATCH (:Season {{season_name:'{season}'}})-[:HAS_GW]->(g)-[:HAS_FIXTURE]->(f)
            MATCH (p:Player)-[pi:PLAYED_IN]->(f)
            WITH p, SUM(pi.{stat}) AS total_stat
            WHERE total_stat > {val}
            RETURN p.player_name AS player, total_stat
            ORDER BY total_stat DESC LIMIT {limit}
        """

    # ----------------------------------------------------------------------
    # 10) FALLBACK
    # ----------------------------------------------------------------------
    return "-- Cannot map to any Cypher query based on extracted intent and entities"

In [39]:
def format_query_result(intent: str, result: list, entities: dict = None) -> str:

    # Handle empty results
    if not result:
        if intent == "fixture_details":
            team = entities.get("team1", "the team") if entities else "the team"
            return f"No fixtures found for {team}."
        elif intent == "best_players_by_metric":
            return "No players found matching the criteria."
        elif intent == "player_or_team_performance":
            entity_name = entities.get("player1") or entities.get("team1") if entities else None
            return f"No performance data found{' for ' + entity_name if entity_name else ''}."
        else:
            return "No results found."
    
    entities = entities or {}
    
    # ========================================================================
    # INTENT 1: FIXTURE_DETAILS
    # ========================================================================
    if intent == "fixture_details":
        if len(result) == 1:
            rec = result[0]
            home = rec.get('h', 'Unknown').get('name', rec.get('home_team', 'Unknown'))
            away = rec.get('a', 'Unknown').get('name', rec.get('away_team', 'Unknown'))
            kickoff = rec.get('f.kickoff_time', 'TBD')
            fixture_num = rec.get('fixture', rec.get('fixture_number', ''))
            
            response = f"**{home} vs {away}**\n"
            if kickoff and kickoff != 'TBD':
                response += f"🕐 Kickoff: {kickoff}\n"
            if fixture_num:
                response += f"📋 Fixture #{fixture_num}"
            return response
        else:
            response = "**Upcoming Fixtures:**\n\n"
            for i, rec in enumerate(result, 1):
                home = rec.get('h', 'Unknown').get('name', rec.get('home_team', 'Unknown'))
                away = rec.get('a', 'Unknown').get('name', rec.get('away_team', 'Unknown'))
                kickoff = rec.get('f.kickoff_time', 'TBD')
                response += f"{i}. {home} vs {away}"
                if kickoff and kickoff != 'TBD':
                    response += f" - {kickoff}"
                response += "\n"
            return response.strip()
    
    # ========================================================================
    # INTENT 2: BEST_PLAYERS_BY_METRIC
    # ========================================================================
    elif intent == "best_players_by_metric":
        stat_type = entities.get("stat_type", "total_points")
        position = entities.get("position")
        limit = entities.get("limit", len(result))
        
        # Convert stat field name to display name
        stat_names = {
            "goals_scored": "Goals", "assists": "Assists", "total_points": "Points",
            "bonus": "Bonus Points", "clean_sheets": "Clean Sheets", 
            "saves": "Saves", "minutes": "Minutes", "form": "Form"
        }
        stat_display = stat_names.get(stat_type, stat_type.replace('_', ' ').title())
        
        position_text = f" {position}" if position else ""
        response = f"**Top{position_text} Players by {stat_display}:**\n\n"
        
        for i, rec in enumerate(result[:limit], 1):
            player = rec.get('player', 'Unknown Player')
            
            # Try different possible field names for the stat value
            stat_value = (rec.get(f'total_{stat_type}') or 
                         rec.get('total_stat') or 
                         rec.get(stat_type) or 
                         rec.get('Total') or 0)
            
            pos = rec.get('position', '')
            pos_text = f" ({pos})" if pos else ""
            
            response += f"{i}. {player}{pos_text}: **{stat_value}** {stat_display}\n"
        
        return response.strip()
    
    # ========================================================================
    # INTENT 3: PLAYER_OR_TEAM_PERFORMANCE
    # ========================================================================
    elif intent == "player_or_team_performance":
        has_player1 = bool(entities.get("player1"))
        has_player2 = bool(entities.get("player2"))
        has_team1 = bool(entities.get("team1"))
        has_team2 = bool(entities.get("team2"))
        has_gw = bool(entities.get("gw_number"))
        
        stat_type = entities.get("stat_type", "total_points")
        stat_names = {
            "goals_scored": "Goals", "assists": "Assists", "total_points": "Points",
            "bonus": "Bonus Points", "clean_sheets": "Clean Sheets", 
            "saves": "Saves", "minutes": "Minutes", "form": "Form"
        }
        stat_display = stat_names.get(stat_type, stat_type.replace('_', ' ').title())
        
        # --------------------------------------------------------------------
        # SUB-CASE 1: Compare Two Players
        # --------------------------------------------------------------------
        if has_player1 and has_player2 and len(result) >= 2:
            player1 = entities["player1"]
            player2 = entities["player2"]
            
            p1_stat = 0
            p2_stat = 0
            
            for rec in result:
                player_name = rec.get('player', '')
                stat_value = rec.get(f'total_{stat_type}', rec.get(stat_type, 0))
                
                if player_name == player1:
                    p1_stat = stat_value
                elif player_name == player2:
                    p2_stat = stat_value
            
            response = f"**Player Comparison - {stat_display}:**\n\n"
            response += f"🔵 {player1}: **{p1_stat}** {stat_display}\n"
            response += f"🔴 {player2}: **{p2_stat}** {stat_display}\n\n"
            
            if p1_stat > p2_stat:
                diff = p1_stat - p2_stat
                response += f"✅ {player1} has {diff} more {stat_display} than {player2}"
            elif p2_stat > p1_stat:
                diff = p2_stat - p1_stat
                response += f"✅ {player2} has {diff} more {stat_display} than {player1}"
            else:
                response += f"⚖️ Both players are equal in {stat_display}"
            
            return response
        
        # --------------------------------------------------------------------
        # SUB-CASE 2: Compare Two Teams
        # --------------------------------------------------------------------
        elif has_team1 and has_team2 and len(result) >= 2:
            team1 = entities["team1"]
            team2 = entities["team2"]
            
            t1_stat = 0
            t2_stat = 0
            
            for rec in result:
                team_name = rec.get('team', '')
                stat_value = rec.get(f'total_{stat_type}', rec.get(stat_type, 0))
                
                if team_name == team1:
                    t1_stat = stat_value
                elif team_name == team2:
                    t2_stat = stat_value
            
            response = f"**Team Comparison - {stat_display}:**\n\n"
            response += f"🏠 {team1}: **{t1_stat}** {stat_display}\n"
            response += f"🏟️ {team2}: **{t2_stat}** {stat_display}\n\n"
            
            if t1_stat > t2_stat:
                diff = t1_stat - t2_stat
                response += f"✅ {team1} has {diff} more {stat_display} than {team2}"
            elif t2_stat > t1_stat:
                diff = t2_stat - t1_stat
                response += f"✅ {team2} has {diff} more {stat_display} than {team1}"
            else:
                response += f"⚖️ Both teams are equal in {stat_display}"
            
            return response
        
        # --------------------------------------------------------------------
        # SUB-CASE 3: Single Player in Specific Gameweek
        # --------------------------------------------------------------------
        elif has_player1 and has_gw and len(result) >= 1:
            rec = result[0]
            player = rec.get('player', entities.get('player1', 'Unknown Player'))
            gw = rec.get('gameweek', entities.get('gw_number', '?'))
            
            stat_value = rec.get(stat_type, rec.get(f'total_{stat_type}', 0))
            
            response = f"**{player} - Gameweek {gw}:**\n\n"
            response += f"⭐ {stat_display}: **{stat_value}**\n"
            
            if 'goals_scored' in rec or 'goals' in rec:
                goals = rec.get('goals_scored', rec.get('goals', 0))
                response += f"⚽ Goals: {goals}\n"
            
            if 'assists' in rec:
                response += f"🎯 Assists: {rec['assists']}\n"
            
            if 'minutes' in rec:
                response += f"⏱️ Minutes: {rec['minutes']}\n"
            
            return response.strip()
        
        # --------------------------------------------------------------------
        # SUB-CASE 4: Single Team in Specific Gameweek
        # --------------------------------------------------------------------
        elif has_team1 and has_gw and len(result) >= 1:
            rec = result[0]
            team = rec.get('team', entities.get('team1', 'Unknown Team'))
            gw = rec.get('gameweek', entities.get('gw_number', '?'))
            
            stat_value = rec.get(f'total_{stat_type}', rec.get(stat_type, 0))
            
            response = f"**{team} - Gameweek {gw}:**\n\n"
            response += f"⭐ Total {stat_display}: **{stat_value}**\n"
            
            return response.strip()
        
        # --------------------------------------------------------------------
        # SUB-CASE 5: Single Player Full Season
        # --------------------------------------------------------------------
        elif has_player1 and not has_gw:
            rec = result[0]
            player = rec.get('player', entities.get('player1', 'Unknown Player'))
            season = entities.get('season', '2022-23')
            
            stat_value = rec.get(f'total_{stat_type}', rec.get(stat_type, 0))
            
            response = f"**{player} - {season} Season:**\n\n"
            response += f"⭐ Total {stat_display}: **{stat_value}**\n"
            
            if 'total_goals_scored' in rec or 'goals' in rec:
                goals = rec.get('total_goals_scored', rec.get('goals', 0))
                response += f"⚽ Goals: {goals}\n"
            
            if 'total_assists' in rec or 'assists' in rec:
                assists = rec.get('total_assists', rec.get('assists', 0))
                response += f"🎯 Assists: {assists}\n"
            
            if 'total_minutes' in rec or 'minutes' in rec:
                minutes = rec.get('total_minutes', rec.get('minutes', 0))
                response += f"⏱️ Minutes: {minutes}\n"
            
            return response.strip()
        
        # --------------------------------------------------------------------
        # SUB-CASE 6: Single Team Full Season
        # --------------------------------------------------------------------
        elif has_team1 and not has_gw:
            rec = result[0]
            team = rec.get('team', entities.get('team1', 'Unknown Team'))
            season = entities.get('season', '2022-23')
            
            stat_value = rec.get(f'total_{stat_type}', rec.get(stat_type, 0))
            
            response = f"**{team} - {season} Season:**\n\n"
            response += f"⭐ Total {stat_display}: **{stat_value}**\n"
            
            return response.strip()
        
        # --------------------------------------------------------------------
        # FALLBACK: Generic Performance
        # --------------------------------------------------------------------
        else:
            response = "**Performance Results:**\n\n"
            
            for i, rec in enumerate(result, 1):
                name = rec.get('player', rec.get('team', f'Entity {i}'))
                
                stat_value = None
                for key in [f'total_{stat_type}', stat_type, 'total_points', 'points']:
                    if key in rec:
                        stat_value = rec[key]
                        break
                
                if stat_value is not None:
                    response += f"{i}. {name}: {stat_value} {stat_display}\n"
                else:
                    response += f"{i}. {name}\n"
            
            return response.strip()
    
    # ========================================================================
    # FALLBACK for unknown intents
    # ========================================================================
    else:
        return "Query executed. Results retrieved."

In [78]:
prompt = "Who are the top 5 forwards by total points in the 2022-23 season?"
intent = classify_fpl_intents(prompt)
print("Classified Intent:", intent)
entities = extract_fpl_entities(prompt)
print("Extracted Entities:", entities)
cypher_query = get_fpl_cypher_query(intent, entities)
print("Cypher Query:\n", cypher_query)
query_result = graph.query(cypher_query)
print("Query Result:\n", query_result)
formatted_response = format_query_result(intent, query_result, entities)
print("Formatted Response:\n", formatted_response)

Classified Intent: best_players_by_metric
Token: who, Lemma: who
Token: are, Lemma: be
Token: the, Lemma: the
Token: top, Lemma: top
Token: 5, Lemma: 5
Token: forwards, Lemma: forwards
Token: by, Lemma: by
Token: total, Lemma: total
Token: points, Lemma: point
Token: in, Lemma: in
Token: the, Lemma: the
Token: 2022, Lemma: 2022
Token: -, Lemma: -
Token: 23, Lemma: 23
Token: season, Lemma: season
Token: ?, Lemma: ?
Extracted Entities: {'stat_type': 'total_points', 'season': '2022-23', 'limit': 5, 'position': 'FWD'}
Cypher Query:
 
            MATCH (p:Player)-[:PLAYS_AS]->(pos:Position {name:'FWD'})
            MATCH (:Season {season_name:'2022-23'})-[:HAS_GW]->(g)-[:HAS_FIXTURE]->(f)
            MATCH (p)-[pi:PLAYED_IN]->(f)
            RETURN p.player_name AS player, SUM(pi.total_points) AS total_total_points, pos.name AS position
            ORDER BY total_total_points DESC LIMIT 5
        
Query Result:
 [{'player': 'Erling Haaland', 'total_total_points': 272, 'position': 'FWD'}, {'

## Embeddings

For creation of feature vector embeddings, text descriptions where constructed from the numerical features using the following query:

MATCH (p:Player)-[:PLAYS_AS]->(pos:Position)
MATCH (p)-[r:PLAYED_IN]->(f:Fixture)<-[:HAS_FIXTURE]-(gw:Gameweek)

// 2. Aggregate ALL stats
WITH p, pos, 
     count(r) as games_played,
     sum(r.total_points) as total_pts,
     sum(r.goals_scored) as goals,
     sum(r.assists) as assists,
     sum(r.minutes) as mins,
     sum(r.clean_sheets) as clean_sheets,
     sum(r.bonus) as bonus,
     sum(r.yellow_cards) as yellow,
     sum(r.red_cards) as red,
     sum(r.saves) as saves,
     round(avg(r.ict_index), 1) as avg_ict,
     round(avg(r.influence), 1) as avg_inf,
     round(avg(r.creativity), 1) as avg_cre,
     round(avg(r.threat), 1) as avg_thr,
     round(avg(r.form), 1) as avg_form

// 3. Construct the Rich Description
// We use descriptive sentences to help the LLM understand the context of the numbers
WITH p, 
     "Player Profile: " + p.player_name + " is a " + pos.name + ". " +
     "Performance: They accumulated " + toString(total_pts) + " total points, " +
     "scoring " + toString(goals) + " goals and providing " + toString(assists) + " assists. " +
     "Advanced Metrics: They had an average ICT Index of " + toString(avg_ict) + 
     " (Influence: " + toString(avg_inf) + ", Creativity: " + toString(avg_cre) + ", Threat: " + toString(avg_thr) + "). " +
     "Defensive & Discipline: They kept " + toString(clean_sheets) + " clean sheets, " +
     "made " + toString(saves) + " saves, and received " + toString(yellow) + " yellow cards. " +
     "Form & Impact: Their average form was " + toString(avg_form) + " and they earned " + toString(bonus) + " bonus points." 
     AS text

// 4. Write to the Player Node
SET p.fpl_features = text
RETURN p.player_name, p.fpl_features LIMIT 3

Then embeddings where created for these texts

In [28]:
def reset_vector_index(index_name, label, property_name, dimension):
    try:
        graph.query(f"DROP INDEX {index_name} IF EXISTS")
    except Exception as e:
        print(f"   - Warning dropping index: {e}")

    # 2. Create the new index
    create_query = f"""
    CREATE VECTOR INDEX {index_name}
    FOR (n:{label})
    ON (n.{property_name})
    OPTIONS {{indexConfig: {{
      `vector.dimensions`: {dimension},
      `vector.similarity_function`: 'cosine'
    }}}}
    """
    
    try:
        graph.query(create_query)
    except Exception as e:
        print(f"   - ❌ Error creating index: {e}")

In [29]:
def generate_feature_vector_embeddings(graph: Neo4jGraph, embedding_model):

    fetch_query = """
    MATCH (p:Player)
    WHERE p.fpl_features IS NOT NULL
    RETURN p.player_name AS name, p.fpl_features AS text
    """
    data = graph.query(fetch_query)

    update_query = """
    MATCH (p:Player {player_name: $name})
    SET p.feature_vector_embedding = $embedding
    """

    for row in data:
        vector = embedding_model.embed_query(row['text'])
        graph.query(update_query, {'name': row['name'], 'embedding': vector})

    reset_vector_index(
        index_name="player_feature_index", 
        label="Player", 
        property_name="feature_vector_embedding", 
        dimension=len(vector)
    )

    print("Feature Vector Embeddings generation complete!")

In [30]:
from langchain_huggingface import HuggingFaceEmbeddings
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
generate_feature_vector_embeddings(graph, embedding_model)

Feature Vector Embeddings generation complete!


In [31]:
from langchain_community.vectorstores import Neo4jVector

def retrieve_embedding_search(query: str, embeddings_model):

    vector_store = Neo4jVector.from_existing_index(
        embedding=embeddings_model,  
        url=config.get('URI'),
        username=config.get('USERNAME'),
        password=config.get('PASSWORD'),
        index_name="player_feature_index",
        node_label="Player",
        embedding_node_property="feature_vector_embedding",
        text_node_property="fpl_features",
    )

    results = vector_store.similarity_search(query, k=3) 
    
    formatted_context = "\n---\n".join([doc.page_content for doc in results])
    
    return formatted_context

In [32]:
from langchain_core.language_models import LLM
from typing import Optional, List, Any
from pydantic import Field

class GemmaLangChainWrapper(LLM):
    client: Any = Field(...)
    max_tokens: int = 500
    
    @property
    def _llm_type(self) -> str:
        return "gemma_hf_api"
    
    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        response = self.client.chat_completion(
            messages=[{"role": "user", "content": prompt}],
            max_tokens=self.max_tokens,
            temperature=0.2 
        )
        return response.choices[0].message["content"]


In [33]:
%pip install -U langchain langchain-community langchain-core pydantic typing-extensions

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [34]:
%pip install langchain langchain-community langchain-core langchain-huggingface

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [35]:
%pip install langchain-classic

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [44]:
from langchain_core.retrievers import BaseRetriever
from langchain_core.documents import Document
from langchain_classic.chains import create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

def rag_pipline(llm, embedding_model, query):
    # 1. Retrieve from KG via Cypher
    intent = classify_fpl_intent(query)
    entities = extract_fpl_entities(query)
    cypher_info = get_fpl_cypher_query(intent, entities)
    cypher_result = graph.query(cypher_info["query"], cypher_info["params"])
    formatted_cypher = format_query_result(intent, cypher_result)

    embedding_context = retrieve_embedding_search(query, embedding_model)

    # 3. Combine Contexts
    combined_context = f"Cypher Results:\n{formatted_cypher}\n\nEmbedding Results:\n{embedding_context}"
    print("Combined Context:\n", combined_context)

    # 4. Create Prompt
    prompt = ChatPromptTemplate.from_template("""
    You are an expert Fantasy Premier League assistant.

    Use the context below to answer the user's question.

    <context>
    {context}
    </context>

    Question: {input}
    """)

    document_chain = create_stuff_documents_chain(llm, prompt)

    class DummyRetriever(BaseRetriever):
        def _get_relevant_documents(self, query: str):
            return [Document(page_content=combined_context)]

    dummy_retriever = DummyRetriever()
    qa_chain = create_retrieval_chain(dummy_retriever, document_chain)

    return qa_chain

In [39]:
from langchain_huggingface import HuggingFaceEndpoint
from huggingface_hub import InferenceClient

HF_TOKEN = config.get('HF_TOKEN')
client = InferenceClient(
        model="google/gemma-2-2b-it",
        token=HF_TOKEN
    )
gemma_llm = GemmaLangChainWrapper(client=client,max_tokens=500)

In [45]:
query = "Who is the top player when it comes to total points in season 2022-23?"
rag_chain = rag_pipline(
    llm=gemma_llm,
    embedding_model=embedding_model,
    query=query
)
response = rag_chain.invoke({"input": query})
print(response["answer"])

Combined Context:
 Cypher Results:
Top Players:
- Erling Haaland: 272 points
- Harry Kane: 263 points
- Mohamed Salah: 239 points
- Martin Ødegaard: 212 points
- Marcus Rashford: 205 points
- Bukayo Saka: 202 points
- Gabriel Martinelli Silva: 198 points
- Kieran Trippier: 198 points
- Kevin De Bruyne: 183 points
- Ivan Toney: 182 points


Embedding Results:
Player Profile: Josh Dasilva is a MID. Performance: They accumulated 81 total points, scoring 4 goals and providing 2 assists. Advanced Metrics: They had an average ICT Index of 1.8 (Influence: 8.0, Creativity: 5.4, Threat: 4.8). Defensive & Discipline: They kept 5 clean sheets, made 0 saves, and received 1 yellow cards. Form & Impact: Their average form was 0.2 and they earned 3 bonus points.
---
Player Profile: Mohamed Naser El Sayed Elneny is a MID. Performance: They accumulated 27 total points, scoring 0 goals and providing 2 assists. Advanced Metrics: They had an average ICT Index of 0.8 (Influence: 3.9, Creativity: 3.3, Threa

In [ ]:
# Test 1: Comparison
user_query_1 = "Who has more goals, Salah or Haaland?"
print(f"Query: {user_query_1}")

ents_1 = extract_fpl_entities(user_query_1)
intent_1 = classify_fpl_intent(user_query_1)
result_1 = get_fpl_cypher_query(intent_1, ents_1)

print(f"Entities: {ents_1}")
print(f"Intent: {intent_1}")
print(f"Cypher Params: {result_1['params']}")
print("-" * 30)

# Test 2: Specific GW Stats
user_query_2 = "How many points did Saka get in Gameweek 5?"
print(f"Query: {user_query_2}")

ents_2 = extract_fpl_entities(user_query_2)
intent_2 = classify_fpl_intent(user_query_2)
result_2 = get_fpl_cypher_query(intent_2, ents_2)

print(f"Entities: {ents_2}")
print(f"Intent: {intent_2}")
print(f"Cypher Params: {result_2['params']}")